* Read community area dataset 
* Conver to JSON

In [1]:
import os
os.chdir("../../")

In [2]:
import json
import pandas as pd

In [3]:
df_areas = pd.read_csv("./pipeline/data/transformed/community_area.csv")

In [4]:
from shapely.geometry import shape, Polygon, MultiPolygon, Point

In [5]:
geom = json.loads(df_areas["geometry"][0])
print("\n".join(json.dumps(geom, indent=2).split("\n")[:20]) + "\n...")

{
  "type": "MultiPolygon",
  "coordinates": [
    [
      [
        [
          -87.65455590025104,
          41.99816614970252
        ],
        [
          -87.65573692713699,
          41.99817520952651
        ],
        [
          -87.65577334679872,
          41.99819571036795
        ],
        [
          -87.65580429073043,
          41.998209147443845
...


In [25]:
df = pd.read_csv("./pipeline/data/extracted/cta_train_stops.csv")

In [50]:
area_multipolygons = [
    (
        area['area_number'], 
        MultiPolygon(shape(json.loads(area['geometry'])))
    ) 
    for area in area_records
]
area_multipolygons

[(1, <shapely.geometry.multipolygon.MultiPolygon at 0x7fdfc181bbd0>),
 (2, <shapely.geometry.multipolygon.MultiPolygon at 0x7fdfc181bc10>),
 (3, <shapely.geometry.multipolygon.MultiPolygon at 0x7fdfc1856cd0>),
 (4, <shapely.geometry.multipolygon.MultiPolygon at 0x7fdfc18567d0>),
 (5, <shapely.geometry.multipolygon.MultiPolygon at 0x7fdfc1856690>),
 (6, <shapely.geometry.multipolygon.MultiPolygon at 0x7fdfc1856650>),
 (7, <shapely.geometry.multipolygon.MultiPolygon at 0x7fdfc1856c10>),
 (8, <shapely.geometry.multipolygon.MultiPolygon at 0x7fdfc18562d0>),
 (9, <shapely.geometry.multipolygon.MultiPolygon at 0x7fdfc17cd2d0>),
 (10, <shapely.geometry.multipolygon.MultiPolygon at 0x7fdfc17cd590>),
 (11, <shapely.geometry.multipolygon.MultiPolygon at 0x7fdfc18561d0>),
 (12, <shapely.geometry.multipolygon.MultiPolygon at 0x7fdfc17ea2d0>),
 (13, <shapely.geometry.multipolygon.MultiPolygon at 0x7fdfc17cd750>),
 (14, <shapely.geometry.multipolygon.MultiPolygon at 0x7fdfc17ea810>),
 (15, <shapely.

In [51]:
def get_community_area(long, lat):
    my_point = Point([long, lat])
    
    for area_num, mp in area_multipolygons:
        if mp.contains(my_point): 
            return area_num
        
    return None

In [63]:
records = df.to_dict(orient = 'records')
area_records = df_areas.to_dict(orient = 'records')

In [64]:
df['area_number'] = [ get_community_area(x['longitude'], x['latitude']) for x in records]

In [67]:
df[df['area_number'].isna()]

,stop_id,direction_id,stop_name,station_name,station_descriptive_name,station_id,ada,red,blue,green,brown,purple,purple_express,yellow,pink,orange,longitude,latitude,area_number
10,30113,E,54th/Cermak (Loop-bound),54th/Cermak,54th/Cermak (Pink Line),40580,True,False,False,False,False,False,False,False,True,False,-87.756692,41.851773,NaN
11,30114,W,54th/Cermak (Terminal arrival),54th/Cermak,54th/Cermak (Pink Line),40580,True,False,False,False,False,False,False,False,True,False,-87.756692,41.851773,NaN
33,30002,W,Austin (Forest Pk-bound),Austin,Austin (Blue Line),40010,False,False,True,False,False,False,False,False,False,False,-87.776812,41.870851,NaN
34,30001,E,Austin (O'Hare-bound),Austin,Austin (Blue Line),40010,False,False,True,False,False,False,False,False,False,False,-87.776812,41.870851,NaN
57,30242,S,Central (Howard-Loop-bound),Central,Central (Purple Line),41250,False,False,False,False,False,True,True,False,False,False,-87.685617,42.063987,NaN
58,30241,N,Central (Linden-bound),Central,Central (Purple Line),41250,False,False,False,False,False,True,True,False,False,False,-87.685617,42.063987,NaN
59,30083,W,Cicero (54th/Cermak-bound),Cicero,Cicero (Pink Line),40420,True,False,False,False,False,False,False,False,True,False,-87.745336,41.851820,NaN
60,30082,E,Cicero (Loop-bound),Cicero,Cicero (Pink Line),40420,True,False,False,False,False,False,False,False,True,False,-87.745336,41.851820,NaN
84,30011,S,Davis (Howard-Loop-bound),Davis,Davis (Purple Line),40050,True,False,False,False,False,True,True,False,False,False,-87.683543,42.047710,NaN
85,30010,N,Davis (Linden-bound),Davis,Davis (Purple Line),40050,True,False,False,False,False,True,True,False,False,False,-87.683543,42.047710,NaN
